# Get 21 cm powerspectrum 
Ref: https://21cmfast.readthedocs.io/en/latest/tutorials/relative_velocities.html

In [ ]:
## Setup py21cmfast package for use

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
# We change the default level of the logger so that
# we can see what's happening with caching.
import logging, sys, os
logger = logging.getLogger('21cmFAST')
logger.setLevel(logging.INFO)

import py21cmfast as p21c

# For plotting the cubes, we use the plotting submodule:
from py21cmfast import plotting

# For interacting with the cache
from py21cmfast import cache_tools

print(f"Using 21cmFAST version {p21c.__version__}")

if not os.path.exists('_cache'):
    os.mkdir('_cache')
    print("created _cache folder")

p21c.config['direc'] = '_cache'
# cache_tools.clear_cache(direc="_cache")
# print("Cache cleared")



Using 21cmFAST version 3.4.0


## Functions to compute power spectrum from brighness temperature

## Run simulation and generate 21cm spectrum

In [10]:
import numpy as np
import collections.abc
#py21cmmc needs the four following aliases to be done manually.
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping
#py21cmmc needs the below
np.int = np.int32
#Now import py21cmmc
from py21cmmc import analyse
from py21cmmc import mcmc
import py21cmmc as p21mc

user_params = {"FAST_FCOLL_TABLES": True, "USE_INTERPOLATION_TABLES": True, "N_THREADS": 6, "USE_FFTW_WISDOM": True}
flag_options = { "USE_MINI_HALOS": True}

coeval = p21c.run_coeval(redshift=9.1, user_params = user_params)
ps = p21mc.Likelihood1DPowerCoeval.compute_power(coeval.brightness_temp, L=100, n_psbins = 10) 
print(ps)

/opt/anaconda3/lib/python3.11/site-packages/powerbox/tools.py:261: FutureWarning: In the future, bins will be generated by default up to the smallest length over any dimension, instead of the largest magnitude for the box.Set bins_upto_boxlen to silence this warning.
  bins = _getbins(bins, coord_mags, log_bins, bins_upto_boxlen=bins_upto_boxlen)


[array([3.50389147e+04, 1.85891216e+04, 9.34564014e+03, 4.25675311e+03,
       1.62527221e+03, 4.44727871e+02, 1.07648152e+02, 2.39123092e+01,
       5.06248781e+00, 2.29944595e+00]), array([0.08130325, 0.13613317, 0.22793973, 0.38165951, 0.63904604,
       1.07001093, 1.79161329, 2.99985552, 5.02292162, 8.41031892])]


In [ ]:
import p21cm_powerspectrum as sim
ps = sim.get_spectra(9.0) 
print(ps)

In [ ]:
import pprint 
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(ps[0])

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(figsize=(6, 3))
plt.plot(ps[0]['k'], ps[0]['delta'], color='#e41a1c')
ax.set_xlabel('k($Mpc^{-1}$)')
ax.set_ylabel('$\Delta^2(mK^2)$')
ax.set_xscale('log')

#fig, axs = plt.subplots(2, 5,
#                         sharex=True, figsize=(14,8))

#for ii in range(10):
#    axs[ii%2][int(ii/2)].plot(ps[ii]['k'], ps[ii]['delta'], color='#e41a1c', linestyle='-')

#plt.tight_layout()
plt.show()


# Generate dataset of powerspectrum
Vary the Astr parameters:       HII\_EFF\_FACTOR (30.0) & ION\_Tvir\_MIN (49999.9995007974)

In [18]:
import numpy as np
import collections.abc
#py21cmmc needs the four following aliases to be done manually.
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping
#py21cmmc needs the below
np.int = np.int32
#Now import py21cmmc
from py21cmmc import analyse
from py21cmmc import mcmc
import py21cmmc as p21mc
import math
import pickle
from datetime import datetime
import time

cache_tools.clear_cache(direc="_cache")
print("Cache cleared")

user_params = {"FAST_FCOLL_TABLES": True, "USE_INTERPOLATION_TABLES": True, "N_THREADS": 6, "USE_FFTW_WISDOM": True}
flag_options = { "USE_MINI_HALOS": True}

filename = datetime.now().strftime("output/ps-%Y%m%d%H%M%S")
print(filename)

zeta_base = 30.0
zeta_low = zeta_base*0.8  # -20%
zeta_high = zeta_base*1.2 # +20%

m_min_base = math.log10(49999.9995007974)
m_min_low = m_min_base+math.log10(0.8) # -20%
m_min_high = m_min_base+math.log10(1.2) # -20%

z = 9.1
nsets = 1000 # number of powerspectra datasets to generate

user_params = {"FAST_FCOLL_TABLES": True, "USE_INTERPOLATION_TABLES": True, "N_THREADS": 6, "USE_FFTW_WISDOM": True}
flag_options = {}# "USE_MINI_HALOS": True}

k_len = -1

start_time = time.time()
for i in range(nsets):
    zeta = np.random.uniform(zeta_low, zeta_high)
    m_min = np.random.uniform(m_min_low, m_min_high)
    astro_params = {   
        "HII_EFF_FACTOR": zeta,
        "ION_Tvir_MIN": m_min
    }
    coeval = p21c.run_coeval(redshift=9.1, user_params = user_params, astro_params=astro_params, flag_options=flag_options)
    ps = p21mc.Likelihood1DPowerCoeval.compute_power(coeval.brightness_temp, L=100, n_psbins = 10) 
    

    # Data validity - skip invalid records
    if (k_len < 0):
        print(ps)
        k_len = len(ps[0])
    elif k_len != len(ps[0]):
        print ("Invalid powerspectrum record: skipping...")
        continue
    ####
    
    with open(filename, 'ab') as f:  # open a text file
        pickle.dump({"zeta": zeta, "m_min": m_min, "ps": ps[0]}, f)
print("--- %s seconds ---" % (time.time() - start_time))


2024-08-21 22:38:04,284 | INFO | Removed 14 files from cache.


Cache cleared
output/ps-20240821223804
[array([1.42056770e+04, 1.00972326e+04, 7.00897690e+03, 3.69422718e+03,
       1.57071752e+03, 4.50481875e+02, 1.13763568e+02, 2.57622723e+01,
       5.38760271e+00, 2.39927614e+00]), array([0.08130325, 0.13613317, 0.22793973, 0.38165951, 0.63904604,
       1.07001093, 1.79161329, 2.99985552, 5.02292162, 8.41031892])]
--- 97.69210410118103 seconds ---


# Test deserialization

In [20]:
import pprint 
pp = pprint.PrettyPrinter(indent=4)
lines = 0
with open('output/ps-20240821223804', 'rb') as f:  # open a text file
    while 1:
        try:
            e = pickle.load(f)
            if(lines < 3):
                pp.pprint(e)
            lines = lines + 1
        except EOFError:
            break
print("--- %d lines ---" % lines)

{   'm_min': 4.766969457811717,
    'ps': array([1.42056770e+04, 1.00972326e+04, 7.00897690e+03, 3.69422718e+03,
       1.57071752e+03, 4.50481875e+02, 1.13763568e+02, 2.57622723e+01,
       5.38760271e+00, 2.39927614e+00]),
    'zeta': 31.34476484466076}
{   'm_min': 4.654987379621536,
    'ps': array([2.24854248e+04, 1.42941062e+04, 9.45037655e+03, 4.63235420e+03,
       1.75182198e+03, 4.49375498e+02, 1.05930943e+02, 2.31065161e+01,
       4.93375259e+00, 2.27561763e+00]),
    'zeta': 29.716626591936382}
{   'm_min': 4.740591937646095,
    'ps': array([1.01151905e+04, 7.54441911e+03, 5.34123062e+03, 2.95433523e+03,
       1.35344415e+03, 4.21886644e+02, 1.09892684e+02, 2.52266943e+01,
       5.29239156e+00, 2.34435546e+00]),
    'zeta': 27.21881946932314}
--- 3 lines ---


# Use py21cmmc to generate powerspectrum dataset

In [ ]:
import numpy as np
import collections.abc
#py21cmmc needs the four following aliases to be done manually.
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping
#Now import py21cmmc
from py21cmmc import analyse
from py21cmmc import mcmc
import py21cmmc as p21mc
import csv
import random
#import argparse
from py21cmfast import cache_tools

#parser = argparse.ArgumentParser()
#parser.add_argument("no_of_sim",help="Num of generations")
#parser.add_argument("f_no",help="File number")
#args=parser.parse_args()

n = 3 #int(args.no_of_sim)
x=3 #int(args.f_no)

#HII_EFF_FACTOR Range:[5,200]
#ION_Tvir_MIN Range:[4,6]
#R_BUBBLE_MAX Range:[5,20]

i=0
while(i<n):
    try:
        h2_eff= random.uniform(5,200) 
        vir_min= random.uniform(4,6)  
        r_mfp= random.uniform(5,20) 

        #Creating Cores
        core = p21mc.CoreCoevalModule( 
            redshift = 9.1,         
            user_params = dict(       
                HII_DIM = 100,        
                BOX_LEN = 100       
            ),
            flag_options={'USE_MASS_DEPENDENT_ZETA': False},
            astro_params={'HII_EFF_FACTOR':h2_eff,'ION_Tvir_MIN':vir_min,'R_BUBBLE_MAX':r_mfp},
            regenerate=False         
        ) 
        
        filepath="output/data_"+str(i+(n*(x-1)))

        datafiles = [filepath+"/simple_mcmc_data_%s.npz"%z for z in core.redshift]
        info_list=[]

        info_list.append([h2_eff,vir_min,r_mfp])

        #Likelihood Function
        likelihood = p21mc.Likelihood1DPowerCoeval(  
            datafile = datafiles,                   
            noisefile= None,                        
            min_k=0.1,                             
            max_k=1.0,                              
            simulate = True,)                    

        model_name = "SimpleTest"

        chain = mcmc.run_mcmc(
            core, likelihood,        # Use lists if multiple cores/likelihoods required. These will be eval'd in order.
            datadir=filepath,          # Directory for all outputs
            model_name=model_name,   # Filename of main chain output
            params=dict(             # Parameter dict as described above.
                HII_EFF_FACTOR = [h2_eff, h2_eff-0.001, h2_eff+0.001, 0.0005],
                ION_Tvir_MIN = [vir_min, vir_min-0.001, vir_min+0.001, 0.0005],
                R_BUBBLE_MAX = [r_mfp, r_mfp-0.001, r_mfp+0.001, 0.0005]
            ),
            walkersRatio=2,         # The number of walkers will be walkersRatio*nparams
            burninIterations=0,      # Number of iterations to save as burnin. Recommended to leave as zero.
            sampleIterations=1,    # Number of iterations to sample, per walker.
            threadCount=2,           # Number of processes to use in MCMC (best as a factor of walkersRatio)
            continue_sampling=False  # Whether to contine sampling from previous run *up to* sampleIterations.
        )


        #Saving the parameters in CSV format
        fields = ['HII_EFF_FACTOR', 'ION_Tvir_MIN','R_BUBBLE_MAX'] 
        with open(filepath+'/data.csv', 'w') as f:
            csv_writer = csv.writer(f)
            csv_writer.writerow(fields)
            csv_writer.writerows(info_list)
        print(i)
        print("Done")
        i=i+1
        cachex=cache_tools.clear_cache()
    except:
        pass
    

/opt/anaconda3/lib/python3.11/site-packages/py21cmfast/inputs.py:515: UserWarning: The USE_INTERPOLATION_TABLES setting has changed in v3.1.2 to be default True. You can likely ignore this warning, but if you relied onhaving USE_INTERPOLATION_TABLES=False by *default*, please set it explicitly. To silence this warning, set it explicitly to True. Thiswarning will be removed in v4.
  warnings.warn(
When attempting to write IonizedBox to file, write failed with the following error. Continuing without caching.
[Errno 28] Can't write data (file write failed: time = Wed Aug 21 19:25:17 2024
, filename = '/Users/sanayaestbelle/21cmFAST-cache/IonizedBox_9585f000d4694187efd71f575eac7e1f_r505344556363.h5', file descriptor = 75, errno = 28, error message = 'No space left on device', buf = 0x7f856a370000, total write size = 4000000, bytes this sub-write = 4000000, bytes actually written = 18446744073709551615, offset = 0)
When attempting to write BrightnessTemp to file, write failed with the follo

[Errno 28] No space left on device


When attempting to write IonizedBox to file, write failed with the following error. Continuing without caching.
[Errno 28] Can't write data (file write failed: time = Wed Aug 21 19:25:21 2024
, filename = '/Users/sanayaestbelle/21cmFAST-cache/IonizedBox_cbe4c928b5304c76bb80d4537af4a6fd_r505344556363.h5', file descriptor = 75, errno = 28, error message = 'No space left on device', buf = 0x7f8539038000, total write size = 4000000, bytes this sub-write = 4000000, bytes actually written = 18446744073709551615, offset = 0)
When attempting to write BrightnessTemp to file, write failed with the following error. Continuing without caching.
[Errno 28] Can't write data (file write failed: time = Wed Aug 21 19:25:21 2024
, filename = '/Users/sanayaestbelle/21cmFAST-cache/BrightnessTemp_2406e40156fea9b45127339a4e8d8dd0_r505344556363.h5', file descriptor = 75, errno = 28, error message = 'No space left on device', buf = 0x7f85384b0000, total write size = 4000000, bytes this sub-write = 4000000, byt

[Errno 28] No space left on device


When attempting to write IonizedBox to file, write failed with the following error. Continuing without caching.
[Errno 28] Unable to create file (file write failed: time = Wed Aug 21 19:25:23 2024
, filename = '/Users/sanayaestbelle/21cmFAST-cache/IonizedBox_a4c142ce8fbdfd869536c57244da4f68_r505344556363.h5', file descriptor = 75, errno = 28, error message = 'No space left on device', buf = 0x600000001980, total write size = 96, bytes this sub-write = 96, bytes actually written = 18446744073709551615, offset = 0)
When attempting to write BrightnessTemp to file, write failed with the following error. Continuing without caching.
[Errno 28] Unable to create file (file write failed: time = Wed Aug 21 19:25:23 2024
, filename = '/Users/sanayaestbelle/21cmFAST-cache/BrightnessTemp_ca3cf4210b630c43dadd9836924fbe10_r505344556363.h5', file descriptor = 75, errno = 28, error message = 'No space left on device', buf = 0x600000030300, total write size = 96, bytes this sub-write = 96, bytes actuall

...

When attempting to write IonizedBox to file, write failed with the following error. Continuing without caching.
[Errno 28] Unable to create file (file write failed: time = Wed Aug 21 19:27:14 2024
, filename = '/Users/sanayaestbelle/21cmFAST-cache/IonizedBox_e653855433e67f6d63e72b165c82d3ce_r505344556363.h5', file descriptor = 75, errno = 28, error message = 'No space left on device', buf = 0x600000030100, total write size = 96, bytes this sub-write = 96, bytes actually written = 18446744073709551615, offset = 0)
When attempting to write BrightnessTemp to file, write failed with the following error. Continuing without caching.
[Errno 28] Unable to create file (file write failed: time = Wed Aug 21 19:27:14 2024
, filename = '/Users/sanayaestbelle/21cmFAST-cache/BrightnessTemp_55c583a74dbb57ecb0cabbda3bbf3a9f_r505344556363.h5', file descriptor = 75, errno = 28, error message = 'No space left on device', buf = 0x600000025600, total write size = 96, bytes this sub-write = 96, bytes actuall

[Errno 28] No space left on device


When attempting to write IonizedBox to file, write failed with the following error. Continuing without caching.
[Errno 28] Can't write data (file write failed: time = Wed Aug 21 19:27:18 2024
, filename = '/Users/sanayaestbelle/21cmFAST-cache/IonizedBox_976c6e9d9fb6939e37e0640efb46a59d_r505344556363.h5', file descriptor = 75, errno = 28, error message = 'No space left on device', buf = 0x7f8569bc0000, total write size = 4000000, bytes this sub-write = 4000000, bytes actually written = 18446744073709551615, offset = 0)
When attempting to write BrightnessTemp to file, write failed with the following error. Continuing without caching.
[Errno 28] Can't write data (file write failed: time = Wed Aug 21 19:27:18 2024
, filename = '/Users/sanayaestbelle/21cmFAST-cache/BrightnessTemp_fe99f2df37ad5a42b553bd4aef247924_r505344556363.h5', file descriptor = 75, errno = 28, error message = 'No space left on device', buf = 0x7f8568888000, total write size = 4000000, bytes this sub-write = 4000000, byt

[Errno 28] No space left on device


When attempting to write IonizedBox to file, write failed with the following error. Continuing without caching.
[Errno 28] Can't write data (file write failed: time = Wed Aug 21 19:28:49 2024
, filename = '/Users/sanayaestbelle/21cmFAST-cache/IonizedBox_31268aadccdede7e2dbe077042b2ec5b_r505344556363.h5', file descriptor = 75, errno = 28, error message = 'No space left on device', buf = 0x7f8538c60000, total write size = 4000000, bytes this sub-write = 4000000, bytes actually written = 18446744073709551615, offset = 0)
When attempting to write BrightnessTemp to file, write failed with the following error. Continuing without caching.
[Errno 28] Can't write data (file write failed: time = Wed Aug 21 19:28:49 2024
, filename = '/Users/sanayaestbelle/21cmFAST-cache/BrightnessTemp_24fb3641a8dfe9bbc0d755a5ff861286_r505344556363.h5', file descriptor = 75, errno = 28, error message = 'No space left on device', buf = 0x7f85684b0000, total write size = 4000000, bytes this sub-write = 4000000, byt